# Portfolio management

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
from plotly import tools
import plotly.graph_objs as go
# from lib import libs as l
from lib import data as d
from lib import plot as p
from lib import finance as f

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

lpalette = {'g':'#13E881', 'r':'#FF5949', 'o':'#FFB84C', 'b':'#A7BEFA', 'w':'#FFFFFF', 'pk':'#FFA1FD', 'bg':'#FFEDBA', 'c':'#90EBF0',
            'pp':'#BBA0F0', 'g2':'#CCFFED', 'b2':'#E3EEFF', 'y':'#FFFC57'}
dpalette = {'g':'#268040', 'r':'#800B00', 'o':'#A13808', 'b':'#464CC2', 'w':'#B8BFBA', 'pk':'#A04EA6', 'bg':'#807155', 'c':'#1D4544',
            'pp':'#291147', 'g2':'#394742', 'b2':'#414449', 'y':'#666523'}

#### Reading data

In [3]:
dquote = pd.read_csv('br_quotes.csv')
dstocks = pd.read_csv('br_stocks.csv')
dnu, dport, dport_inv, dprices = f.read_invest()

dflow = dnu.groupby('ID').apply(f.calc_flows)
dflow = dflow.reset_index(level=[0, 1])
dflow.pop('level_1')
dflow = dflow.sort_values(by=['Rendimento anual'], ascending=False).reset_index(drop=True)

#### Calculating stocks return

In [4]:
dreturn = pd.DataFrame()
current_year = dt.date.today().year
dreturn['Ticker'] = dquote['Ticker']
dreturn['Predicted'] = [0 for j in dquote['Ticker']]
weights = [0.05, 0.1, 0.15, 0.3, 0.4]
for i in range(5):
    year_i = current_year-4+i
    dreturn[str(year_i)] = (dquote[str(year_i)] - dquote[str(year_i-1)]) / dquote[str(year_i-1)] 
    dreturn[str(year_i)] = dreturn[str(year_i)].apply(lambda x : 0 if pd.isnull(x) or x == np.inf else x)
    dreturn['Predicted'] += dreturn[str(year_i)] * weights[i]

# Screener

In [5]:
assets = ['USA', 'EU', 'BR', 'EM', 'Bonds', 'Cash', 'Commod']
industry = ['Basic Mat', 'Cons Cycl', 'Finance', 'Real Estate', 'Telecom', 'Energy', 'Indust', 'Tech', 'Cons Non-Cycl', 'Health', 'Util']

dfull = f.calc_stocks_feat(dreturn, dstocks, dquote, assets, industry)

screen = ['Ticker', 'Score', 'Price', 'Return', 'Risk', 'Tax', 'YTD', 'Cat', 'Geo', 'Domain', 'Asset']
p.change_df_prop(dfull[screen].sort_values('Score', ascending=False).reset_index(drop=True))

Ticker,Score,Price,Return,Risk,Tax,YTD,Cat,Geo,Domain,Asset
IVVB11,6.79437,147.72,17.52%,12.18%,0.24%,6.5%,2,USA,Tech,Equity
Cash,6.10837,6.7,0.0%,0.0%,0.0%,0.0%,0,Cash,Basic Mat,Cash
DIVO11,5.64439,63,16.92%,19.25%,0.5%,-9.35%,3,USA,Util,Equity
BBSD11,5.52245,83,15.02%,17.73%,0.5%,-13.3%,3,USA,Indust,Equity
BRAX11,4.98306,82,10.55%,17.38%,0.2%,-14.45%,1,BR,Finance,Equity
BOVB11,4.72318,98.06,9.49%,17.73%,0.2%,-15.24%,1,BR,Finance,Equity
BOVV11,4.70211,98.52,9.67%,17.73%,0.3%,-15.25%,1,BR,Finance,Equity
PIBB11,4.51295,163.87,8.49%,17.68%,0.06%,-16.03%,1.1,BR,Finance,Equity
GOVE11,4.27892,43,11.52%,22.55%,0.5%,-14.68%,1,BR,Finance,Equity
MATB11,3.69305,31.95,7.88%,22.55%,0.5%,-11.64%,4,BR,Basic Mat,Equity


# Portfolio Overview

In [6]:
dport_full = dport.join(dfull[['Ticker', 'Price', 'TER'] + assets + industry].set_index('Ticker'), on='Ticker')
dport_full = f.calc_portfolio(dport_full)
dport_bonds = f.add_bonds_to_port(dport_full, dflow, assets)
dport_bonds = f.calc_distribution(dport_bonds, assets, industry)

## Current state

In [7]:
dtotal = pd.DataFrame({
    'Value': [(dport_full['Price'] * dport_full['Shares']).sum()],
    'Profit': [dport_full['Profit'].sum()],
    'Yield': [d.int2pct(dport_full['Profit'].sum() / (dport_full['Buy'] * dport_full['Shares']).sum())]
})
dtotal['Year profit'] = dport_full['Year profit'].sum()
dtotal['Annual yield'] = (dtotal['Year profit'] / dtotal['Value']).apply(d.int2pct)
p.change_df_prop(dtotal, 28)

Value,Profit,Yield,Year profit,Annual yield
9388.3,-422.1,-4.3%,-2529.42,-26.94%


## Portfolio components

In [8]:
status = ['Ticker', 'Value', 'Profit', 'Yield %']
p.change_df_prop(dport_bonds[status], 24)

Ticker,Value,Profit,Yield %
IVVB11,1477.2,169.2,12.94%
BRAX11,820,-55,-6.29%
IVVB11,1477.2,86.9,6.25%
BBSD11,830,-127.2,-13.29%
BRAX11,820,-150.3,-15.49%
IVVB11,1477.2,-0.8,-0.05%
BBSD11,830,-136.7,-14.14%
BRAX11,820,-144,-14.94%
BBSD11,830,-64.2,-7.18%
Cash,6.7,0,0.0%


In [9]:
revenue = ['Ticker', 'Annual yield %', 'Year profit', 'Tax', 'Annual yield (Liq.) %', 'Year profit (Liq.)']
p.change_df_prop(dport_full[revenue], 24)

Ticker,Annual yield %,Year profit,Tax,Annual yield (Liq.) %,Year profit (Liq.)
IVVB11,41.09%,606.926,16.98%,33.46%,494.252
BRAX11,-18.77%,-153.895,12.05%,-16.65%,-136.506
IVVB11,29.35%,433.498,19.08%,23.29%,344.101
BBSD11,-59.23%,-491.639,12.03%,-54.27%,-450.404
BRAX11,-70.02%,-574.131,13.85%,-64.12%,-525.806
IVVB11,-0.55%,-8.0867,-409.5%,-2.76%,-40.787
BBSD11,-85.32%,-708.185,12.23%,-81.14%,-673.441
BRAX11,-98.05%,-804.002,13.86%,-96.5%,-791.277
BBSD11,-99.99%,-829.904,9.03%,-99.97%,-829.776
Cash,0.0%,0,0.0%,0.0%,0


## Asset allocation

In [10]:
fig_asset = p.make_fig(cols=2, specs=[[{},{}]])

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [11]:
palette = {'BR': lpalette['g'], 'Cash': lpalette['c'], 'Bonds': dpalette['b'], 'EU': lpalette['pp'], 'USA': lpalette['r'], 'EM':lpalette['pk'],
           'Finance': lpalette['g'], 'Indust': lpalette['c'], 'Tech': dpalette['b'], 'Cons Non-Cycl': lpalette['pp'], 'Cons Cycl': lpalette['r'], 'Health':lpalette['pk'],
           'Energy': dpalette['g'], 'Util': dpalette['c'], 'Basic Mat': lpalette['b'], 'Real Estate': dpalette['pp'], 'Telecom': dpalette['r'], 'Commod':lpalette['pk']
          }
dgroup = dport_bonds.sum()
p.plot_pie(dgroup[assets], fig_asset, palette, dict(x=[0, 0.5]))
p.plot_pie(dgroup[industry], fig_asset, palette, dict(x=[0.5, 1]))

# fig = l.plot_pie(dgroup[assets], domain=dict(x=[0,0.5]), name='Asset')
# l.plot_pie(dgroup[industry], fig=fig, domain=dict(x=[0.5,1]), name='Industry')

FigureWidget({
    'data': [{'domain': {'x': [0, 0.5]},
              'hole': 0.4,
              'labels': arr…

# Portfolio Evolution

In [12]:
dports = f.calc_port_months(dport, dfull, dprices, assets, industry)

In [13]:
dbonds = f.calc_bonds_months(dprices, dnu, assets, industry)
dports_full = pd.concat([dports, dbonds], sort=False)
for dd in dports_full['Date'].unique():
    di = dports_full['Date'] == dd
    dports_full.loc[di,:] = f.calc_distribution(dports_full.loc[di, :], assets, industry)

D:\Office files\Personal_Financial_Manager\lib\finance.py:228: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

D:\Office files\Personal_Financial_Manager\lib\finance.py:230: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



## Actives allocation

In [14]:
fig_act = p.make_fig()
fig_act_pct = p.make_fig(cols=3, specs=[[{'colspan':2},{},{}]])

This is the format of your plot grid:
[ (1,1) x1,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1    [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]



In [15]:
palette = {'BRAX11': lpalette['g'], 'Cash': lpalette['c'], 'NU': dpalette['b'], 'BBSD11': lpalette['pp'], 'IVVB11': lpalette['r'], 'Portfolio': lpalette['b2']}
devol = d.pivot_agg(dports_full, val='Value', idx=['Date'], date='Date', col=['Ticker'], agg={'Value':np.sum})
p.plot(devol, 'Date', fig_act, palette, plot_type='area')
# p.plot(d.row_percentage(devol.set_index('Date')).reset_index(), 'Date', fig_act, palette, col=2, plot_type='area')

FigureWidget({
    'data': [{'hoverinfo': 'name+y',
              'marker': {'color': '#BBA0F0'},
            …

In [16]:
dgroup = dport_bonds.groupby('Ticker').sum()
p.plot_pie(dgroup['Value'], fig_act_pct, palette, dict(x=[0.68, 1]))
# p.plot_pie(dgroup['Profit'], fig_actives, palette, dict(x=[0.5, 1]))
p.plot(d.row_percentage(devol.set_index('Date')).reset_index(), 'Date', fig_act_pct, palette, col=1, plot_type='area')

FigureWidget({
    'data': [{'domain': {'x': [0.68, 1]},
              'hole': 0.4,
              'labels': ar…

## Profits

In [17]:
fig_tot = p.make_fig(cols=3, specs=[[{'colspan':2},{},{}]])
fig_prof = p.make_fig()
fig_yield = p.make_fig(cols=2, specs=[[{}, {}]])

This is the format of your plot grid:
[ (1,1) x1,y1    [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



### Monthly profit

In [18]:
palette2 = {'BRAX11Avg': dpalette['g'], 'NUAvg': lpalette['b'], 'BBSD11Avg': dpalette['pp'], 'IVVB11Avg': dpalette['r']}
devol = d.pivot_agg(dports_full, val='Profit', idx=['Date'], date='Date', col=['Ticker'], agg={'Profit':np.sum})
devol = devol[['Date', 'BRAX11', 'NU', 'BBSD11', 'IVVB11']]
for c in devol.columns:
    if c == 'Date':
        continue
    devol = d.window(devol, val=c)
    p.plot(devol, 'Date', fig_prof, palette2, row=1, col=1, y=[c + 'Avg'], name=[c + ' Avg'])
    p.plot(devol, 'Date', fig_prof, palette, row=1, col=1, y=[c], plot_type='bar', name=[c])
fig_prof

FigureWidget({
    'data': [{'hoverinfo': 'name+y',
              'marker': {'color': '#268040'},
            …

### Total profit

In [19]:
temp_sum = devol[['Date', 'BRAX11', 'NU', 'BBSD11', 'IVVB11']].sum()
devol_sum = temp_sum.drop(temp_sum.index[0])
p.plot_pie(devol_sum, fig_tot, palette, dict(x=[0.68, 1]))
devol = d.pivot_agg(dports_full, val='Total profit', idx=['Date'], date='Date', col=['Ticker'], agg={'Total profit':np.sum})
devol.pop('Cash')
p.plot(devol, 'Date', fig_tot, palette, plot_type='area')

FigureWidget({
    'data': [{'domain': {'x': [0.68, 1]},
              'hole': 0.4,
              'labels': ar…

### Yield

In [20]:
devol_val = d.pivot_agg(dports_full, val='Value', idx=['Date'], date='Date', col=['Ticker'], agg={'Value':np.sum})
date = devol_val.pop('Date')
devol_prof = d.pivot_agg(dports_full, val='Profit', idx=['Date'], date='Date', col=['Ticker'], agg={'Profit':np.sum})
date = devol_prof.pop('Date')
devol = devol_prof.div(devol_val) * 100
devol.pop('Cash')
devol['Date'] = date
for c in devol.columns:
    if c  == 'Date':
        continue
    p.plot(devol, 'Date', fig_yield, palette, col=1, y=[c], name=[c])

devol = dports_full[dports_full['Ticker'] != 'NU'].groupby(['Date', 'Ticker']).apply(lambda x : pd.DataFrame([[x['Initial'].sum(), x['Value'].sum(), x['Profit'].sum(), x['Total profit'].sum()]], 
                                                                      columns=['Initial', 'Value', 'Profit', 'Total profit'])).reset_index()
devol.pop('level_2')
dtotal = dports_full[dports_full['Ticker'] != 'NU'].groupby(['Date']).apply(lambda x : pd.DataFrame([[x['Initial'].sum(), x['Value'].sum(), x['Profit'].sum(), x['Total profit'].sum()]], 
                                                                      columns=['Initial', 'Value', 'Profit', 'Total profit'])).reset_index()
dtotal.pop('level_1')
dtotal['Ticker'] = 'Portfolio'
devol = devol.append(dtotal, sort=False)
devol['Yield'] = devol['Total profit'] / devol['Initial'] * 100
# devol['Total yield'] = devol['Total profit'] / devol['Initial']
devol = d.pivot_agg(devol, val='Yield', idx=['Date'], date='Date', col=['Ticker'], agg={'Yield':np.sum})
# devol = l.group_sum(devol, select='Yield', group=['Ticker'], date_col='Date')
devol.pop('Cash')
devol
for c in devol.columns:
    if c  == 'Date':
        continue
    p.plot(devol, 'Date', fig_yield, palette, col=2, y=[c], name=[c])
# fig_tot_yield

fig_yield

FigureWidget({
    'data': [{'hoverinfo': 'name+y',
              'marker': {'color': '#BBA0F0'},
            …